# Homework 3

## By Shehzad Ali, Bhavana Reddy, and Lucas Fernandez

In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


<b>Q1.)[10 Points] </b> Read in the files gold.txt and labels.txt. The gold DataFrame should have columns url and category, while the labels DataFrame should have columns turk, url and category. You will have to pick the right separator.

In [2]:
#Create gold and labels dataframes and seperate data on tabs.
gold = pd.read_csv('gold.txt',
                      sep='\t',      
                      header=None,   
                      names=['url', 'category'])
labels = pd.read_csv('labels.txt',
                      sep='\t',      
                      header=None,   
                      names=['turk','url', 'category'])

In [3]:
gold[:5]

,url,category
0,http://0800-horoscope.com,G
1,http://18games.net,X
2,http://1pixelout.net,G
3,http://1porno.com/004/teens/index.html,X
4,http://1stpussy.com/005/16/index.html,X


In [4]:
labels[:5]

,turk,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G


<b>Q2.)[10 Points] </b> Split the labels DataFrame into two: labels_on_gold and labels_unknown, the former containing all rows where the url is present in the gold set, and the latter one contains all remaining rows of labels.

In [5]:
#Merge the two dataframes on url to create a dataframe with all urls that are in gold.
#Then create a dataframe with rows of data from labels that were not in gold. 
labels_on_gold = labels.merge(gold,
                            on = 'url',
                             suffixes=['_labels', '_gold'])
labels_unknown = labels[~labels.url.isin(labels_on_gold.url)]

In [6]:
labels_on_gold[:5]

,turk,url,category_labels,category_gold
0,A1253FXHCZ9CWM,http://0800-horoscope.com,G,G
1,A153PKAL7OAY36,http://0800-horoscope.com,G,G
2,A1FV9SAPL5C6KY,http://0800-horoscope.com,G,G
3,A1JTOT0DWM6QGL,http://0800-horoscope.com,G,G
4,A1PXXEOGQ76RNJ,http://0800-horoscope.com,G,G


In [7]:
labels_unknown[:5] 
#The data has duplicate entries, but we decided to keep the duplicate rows because a turk can review the same website multiple
#times. We also noticed that the url can have the same website with different capitalization, but we decided to focus on 
#keeping the data as is for the solution. 

,turk,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G


<b>Q3.)[10 Points] </b> Create a rater_goodness DataFrame that is indexed by turk, and has two columns: the number of ratings, and the average correctness of ratings for each turk (both on gold set urls).

In [8]:
def rating_diff(correct_label):
    """Essentially makes a dummy variable that assess whether the turks was correct or not"""
    
    if correct_label['category_labels'] == correct_label['category_gold']:
        correct = 1
    else:
        correct = 0
    return correct

In [9]:
#Run the rating difference function on the labels_on_gold and create a new column that has that data. 
rating_differences = labels_on_gold.T.apply(rating_diff)
labels_on_gold["SameCategory"] = rating_differences
#Group data by turk to make it the index and use agg to get count and mean
#Mean will add up the values and divide by count giving it the average correctness
rater_goodness = labels_on_gold.groupby("turk")["SameCategory"].agg(["mean","count"])
rater_goodness

,mean,count
turk,,
A112DVP1KG4QZU,1.000000,1
A1253FXHCZ9CWM,0.517241,29
A12CY1Q7XKJJDE,1.000000,1
A12RE8G66WTO8B,0.750000,20
A12Y1GTGIQDGRA,0.333333,3
...,...,...
AYWOBMTEGFVVI,0.000000,1
AZ1S5BIG5R6P6,0.714286,35
AZSDW7JJL5VA7,0.680000,100


<b>Q4.)[10 Points] </b> Someone is correct p fraction of the time, the odds of success are defined as: odds = p / (1.001 - p)
Attach a column called odds to the rater goodness DataFrame, using the average correctness of the turk as his or her p.

In [10]:
#Use the formula given in the question to create the Odds column
rater_goodness["Odds"] = rater_goodness["mean"]/(1.001-rater_goodness["mean"])
rater_goodness

,mean,count,Odds
turk,,,
A112DVP1KG4QZU,1.000000,1,1000.000000
A1253FXHCZ9CWM,0.517241,29,1.069214
A12CY1Q7XKJJDE,1.000000,1,1000.000000
A12RE8G66WTO8B,0.750000,20,2.988048
A12Y1GTGIQDGRA,0.333333,3,0.499251
...,...,...,...
AYWOBMTEGFVVI,0.000000,1,0.000000
AZ1S5BIG5R6P6,0.714286,35,2.491281
AZSDW7JJL5VA7,0.680000,100,2.118380


<b>Q5.)[10 Points] </b> List the 10 most accurate turks who have rated at least 20 gold set URLs

In [11]:
#Create a mask for all the URL counts greater than or equal to 20 then sort the data. 
accurateTurks20 = rater_goodness[rater_goodness["count"] >= 20].sort_values(by = "Odds", ascending = False)
accurateTurks20[:10]

,mean,count,Odds
turk,,,
A2U0R4X38GUKZE,0.950000,20,18.627451
A22C0PJUBFJTI0,0.916667,36,10.869565
A23YQUBXZPKILZ,0.875000,24,6.944444
ATVALOQVDCMZW,0.854369,103,5.826657
A1HIXWH4OXT8S4,0.825000,40,4.687500
A3220HG1O83HQ4,0.818182,22,4.475385
A32W20KGQXS0LL,0.800000,25,3.980100
A20PWAB7G3HDHU,0.800000,20,3.980100
AJSJVK40F5HM6,0.785714,28,3.649635


<b>Q6.)[10 Points] </b> One may imagine that a committed and accurate turk will rate lots of URLs. On the other hand, perhaps it is only the spammers who have the time to rate lots of URLs. Is number of ratings by a turker on gold set URLs related to his or her
accuracy? There's no fixed answer; just try to show some evidence for your answer.

In [12]:
#There's no set answer for this one. So what I am going to look at is compare the top 10% of turks (26 turks of the 269)
#in terms of turks that have rated gold set url's. Then I am going to compute their combined accuracy and compare that to the 
#combined accuracy of all of the other turks. We'll say there is a difference if the accuracies are off by more than 0.1
#I could do statistical analysis involving p-values and norm distribution, but I think we're just trying to get a general sense
#of accuracy
accuracyCheck = rater_goodness.sort_values(by = "count", ascending = False)
accuracyCheck["CorrectUrlPerTurk"] = accuracyCheck["mean"] * accuracyCheck["count"]
top10Perc = sum(accuracyCheck.iloc[0:26]['CorrectUrlPerTurk'])/sum(accuracyCheck.iloc[0:26]['count'])
bott90Perc = sum(accuracyCheck.iloc[26:]['CorrectUrlPerTurk'])/sum(accuracyCheck.iloc[26:]['count'])
print(f'The top ten percent of turks have an accuracy of {round(top10Perc,4)}')
print(f'The bottom 90% of turks have an accuracy of {round(bott90Perc,4)}')
print('There is only a small difference in the accuracy of a turk with lots of rating and a turk with fewer ratings')

The top ten percent of turks have an accuracy of 0.6698
The bottom 90% of turks have an accuracy of 0.6959
There is only a small difference in the accuracy of a turk with lots of rating and a turk with fewer ratings


In [13]:
#What about when looking at turks with atleast 3 responses to filter out low count turks with high percentage correct.
accurateTurks3 = rater_goodness[rater_goodness["count"] >= 3].sort_values(by = "count", ascending = False)
len(accurateTurks3) #142
accurateTurks3["CorrectUrlPerTurk"] = accurateTurks3["mean"] * accurateTurks3["count"]
top10PercFiltered = sum(accurateTurks3.iloc[0:14]['CorrectUrlPerTurk'])/sum(accurateTurks3.iloc[0:14]['count'])
bott90PercFiltered = sum(accurateTurks3.iloc[14:]['CorrectUrlPerTurk'])/sum(accurateTurks3.iloc[14:]['count'])
print(f'The top ten percent of turks have an accuracy of {round(top10PercFiltered,4)}')
print(f'The bottom 90% of turks(min 3 responses) have an accuracy of {round(bott90PercFiltered,4)}')
print('We were curious how the turks who answered 1-2 questions and had a 100% mean were affecting the data. When we took them out and recalibrated our top 10% and bottom 90%, we saw that Turks who answer the most questions have higher accuracy than than spammers')

The top ten percent of turks have an accuracy of 0.7094
The bottom 90% of turks(min 3 responses) have an accuracy of 0.6663
We were curious how the turks who answered 1-2 questions and had a 100% mean were affecting the data. When we took them out and recalibrated our top 10% and bottom 90%, we saw that Turks who answer the most questions have higher accuracy than than spammers


<b>Q7.)[13 Points] </b> Consider each url u that is not in the gold set, and each category c. For the pair (u; c), calculate the product of odds of all turks who (a) rated url u as category c, and (b) have rated more gold set urls than 75% of all turks who rated at least one gold-set url.

For example, if you find that there are 269 turks who rated at least one gold-set url, you want to select only the turks who have rated more gold-set urls than 75% of these 269 turks. We can think of these as our "reliable" turks. Now, our belief that url u belongs to category c depends on how many "reliable" turks rated u as c; specifically, our belief is based on the product of their reliability scores (i.e., their odds).

We shall call such products of odds the overall odds henceforth.

In [14]:
#Once you reset the index once, you don't need to do it multiple times. So Once I did it 
labels_unknown.reset_index(inplace = True)
rater_goodness.reset_index(inplace = True)
accuracyCheck.reset_index(inplace = True)
s = rater_goodness['count'].quantile(q=0.75) 
mask = rater_goodness['count'] >= s
turks_temp = rater_goodness[mask] 
turks_reliable = turks_temp[['turk','Odds']].reset_index()

labels_unknown75 = pd.merge(turks_reliable,labels_unknown, on = 'turk')
labels_predict = labels_unknown75.groupby(['url','category']).prod().reset_index().rename(columns = {'Odds':'Overall Odds'})
labels_predict[['url','category','Overall Odds']]

,url,category,Overall Odds
0,http://0-101.net,G,2.155963
1,http://000.cc,G,1.460583
2,http://0000.jp,G,14.488244
3,http://000relationships.com,G,9.914590
4,http://000relationships.com,P,1.851852
...,...,...,...
14109,http://zynga.com,G,3.704611
14110,http://zz868.com,G,1.047852
14111,http://zz868.com,P,1.851852
14112,http://zzx.cc,G,1.330229


<b>Q8.)[13 Points] </b> Create a DataFrame (called result 75, whose index is URLs not in the gold set, and with two columns called top category and top odds. The top category should be the category with the highest overall odds for that url, and the top odds should be the overall odds for that top category. These are our predictions, and the confidence we have in them (higher overall odds implies greater confidence). If you want, you can check to see if the predicted categories make sense.

In [15]:
#Transforming the data so that each every url only has a repeating list of the max value for overall odds
maxOdds = labels_predict.groupby(['url'])['Overall Odds'].transform('max')

#then I make a mask essentially asking if the value of the overall odds is equal to the max
maskMax = labels_predict['Overall Odds'] == maxOdds
result_75 = labels_predict[maskMax].rename(columns = {'category':'Top Category','Overall Odds':'Top Odds'})
result_75.set_index("url")[["Top Category","Top Odds"]]

,Top Category,Top Odds
url,,
http://0-101.net,G,2.155963
http://000.cc,G,1.460583
http://0000.jp,G,14.488244
http://000relationships.com,G,9.914590
http://000vitamins.com,G,3.784982
...,...,...
http://zwinky.com,G,16.141443
http://zylom.com,G,1.735526
http://zynga.com,G,3.704611


<b>Q9.)[14 Points] </b> Questions 7 and 8 above only considered the ratings of turks who had rated enough gold set URLs, so we were relatively more confident about their accuracies. What happens if we loosen this restriction?

Repeat the code of Q7 and Q8, but replacing 75% by 25% in the description of Q7 (i.e., we also consider turks who have far fewer gold set ratings).
Call this result 25.

Now let's see how these two results compare. Create a DataFrame where both the index and the columns are the various categories, and the cells contain the number of urls with these as the top categories according to result 75 and result 25.

For example, the cell corresponding to the row category=R and the column category=G would be the number of URLs that were predicted to be R by result 75 but predicted to be G by result 25.

Where are the most errors? The top category should be the category with the highest overall odds for that url, and the top odds should be the overall odds for that top category. These are our predictions, and the confidence we have in them (higher overall odds implies greater confidence). If you want, you can check to see if the predicted categories make sense.

In [16]:
#essentially copying and pasting for the top 75% quartile now

s25 = rater_goodness['count'].quantile(q=0.25) 
#s25 = 1. so for our mask we are trying to get greater than 1. If we chose to get >= s25, then we would get all values. 
mask25 = rater_goodness['count'] > s25
turks_temp25 = rater_goodness[mask25] 
turks_reliable25 = turks_temp25[['turk','Odds']].reset_index()

labels_unknown25 = pd.merge(turks_reliable25,labels_unknown, on = 'turk')
labels_predict25 = labels_unknown25.groupby(['url','category']).prod().reset_index().rename(columns = {'Odds':'Overall Odds'})
labels_predict25[['url','category','Overall Odds']]

maxOdds = labels_predict25.groupby(['url'])['Overall Odds'].transform('max')


#then I make a mask essentially asking if the value of the overall odds is equal to the max
mask25Odds = labels_predict25['Overall Odds'] == maxOdds
result25 = labels_predict25[mask25Odds].rename(columns = {'category':'Top Category','Overall Odds':'Top Odds'})

#I make a merged dataframe of the two results so I can compare them
merged_resultsDF = result_75.merge(result25, left_on='url', right_on='url',suffixes = ["_75","_25"])
merged_resultsCT = pd.crosstab(merged_resultsDF['Top Category_25'],merged_resultsDF['Top Category_75'])
#crosstable just to check that it matches my pivot table
print(merged_resultsCT)

#this is the dataframe of the pivot table results. It is kinda weird how there has to be url column included in the
#pivot table call. I am not sure why this is and it could not pivot off the original index (maybe since it did not have a name?)
results_check = merged_resultsDF[['url','Top Category_25','Top Category_75']].pivot_table(merged_resultsDF[['url','Top Category_25','Top Category_75']], index='Top Category_25', columns='Top Category_75', aggfunc='count')
print('')
print(results_check)
print("The most errors are Top Category_25 predicted P and Top Category_75 predicted G")

Top Category_75     G    P    R    X
Top Category_25                     
G                8329  181   21   34
P                 564  336   36    9
R                 177   53  130   29
X                 203   15   28  474

                  url               
Top Category_75     G    P    R    X
Top Category_25                     
G                8329  181   21   34
P                 564  336   36    9
R                 177   53  130   29
X                 203   15   28  474
The most errors are Top Category_25 predicted P and Top Category_75 predicted G
